In [19]:
import librosa
import os
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
from google.colab import drive
from google.colab import files
import zipfile
import pandas as pd
import cv2
import keras
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import shutil
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

In [3]:
def zip_list(file_path):
    zf = zipfile.ZipFile(file_path, 'r')
    zf.extractall()



file_path = '/content/drive/MyDrive/voice_dataset.zip'
zip_list(file_path)

In [4]:
OutputFolder = '/content/voice'
# 新建檔案
os.mkdir(OutputFolder)

In [5]:

ImageList = os.listdir('/content/voice_dataset/train_set')
ImageList.sort()
ImageList.sort(key=lambda x:int(x[:-4]))
#ImageList = [img for img in ImageList if len(img)>1]
WordList = ['0','1','2','3','4','5','6','7']
c = 0
sr = pd.read_csv('/content/train.csv')

word = (list(sr["label"]))

n = []
for a in word :
  n.append(str(a))
  

for w in WordList:
  try:
    os.chdir(OutputFolder) # 指定當前工作目錄
    os.mkdir(w) # 創建字名資料夾
    #MoveList = [img for img in ImageList if w in img]
                
  except: 
    os.chdir(OutputFolder) # 已有字名資料夾
    #MoveList = [ img for img in ImageList if w in img ]
  



# 移動所有影像檔到對應字名資料夾  
for img in ImageList:
   old_path = '/content/voice_dataset/train_set' + '/' + img
   new_path = OutputFolder + '/' + n[c] + '/' + img
   c += 1 
   shutil.move(old_path, new_path)



In [6]:
DATA_PATH = "/content/voice"


# Input: Folder Path
# Output: Tuple (Label, Indices of the labels, one-hot encoded labels)
def get_labels(path=DATA_PATH):
    labels = os.listdir(path)
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, tf.keras.utils.to_categorical(label_indices)


# Handy function to convert wav2mfcc
def wav2mfcc(file_path, max_len=11):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    wave = wave[::3]
    mfcc = librosa.feature.mfcc(wave, sr=16000)

    # If maximum length exceeds mfcc lengths then pad the remaining ones
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

    # Else cutoff the remaining parts
    else:
        mfcc = mfcc[:, :max_len]
    
    return mfcc


def save_data_to_array(path=DATA_PATH, max_len=11):
    labels, _, _ = get_labels(path)
    os.chdir('/content/voice')
    for label in labels:
        # Init mfcc vectors
        mfcc_vectors = []

        wavfiles = [path + '/' +label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]
        for wavfile in tqdm(wavfiles, "Saving vectors of label - '{}'".format(label)):
            mfcc = wav2mfcc(wavfile, max_len=max_len)
            mfcc_vectors.append(mfcc)
            os.chdir('/content')
        np.save(label + '.npy', mfcc_vectors)
        os.chdir('/content/voice')

def get_train_test(split_ratio=0.6, random_state=42):
    # Get available labels
    os.chdir('/content')
    labels, indices, _ = get_labels(DATA_PATH)

    # Getting first arrays
    X = np.load(labels[0] + '.npy')
    y = np.zeros(X.shape[0])

    # Append all of the dataset into one single array, same goes for y
    for i, label in enumerate(labels[1:]):
        x = np.load(label + '.npy')
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value= (i + 1)))

    assert X.shape[0] == len(y)

    return train_test_split(X, y, test_size= (1 - split_ratio), random_state=random_state, shuffle=True)



def prepare_dataset(path=DATA_PATH):
  
    labels, _, _ = get_labels(path)
    data = {}
    for label in labels:
        data[label] = {}
        data[label]['path'] = [path  + '/' +label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]

        vectors = []

        for wavfile in data[label]['path']:
            wave, sr = librosa.load(wavfile, mono=True, sr=None)
            # Downsampling
            wave = wave[::3]
            mfcc = librosa.feature.mfcc(wave, sr=16000)
            vectors.append(mfcc)

        data[label]['mfcc'] = vectors

    return data


def load_dataset(path=DATA_PATH):
    data = prepare_dataset(path)

    dataset = []

    for key in data:
        for mfcc in data[key]['mfcc']:
            dataset.append((key, mfcc))

    return dataset[:100]


#print(prepare_dataset(DATA_PATH))

In [7]:
feature_dim_2 = 11
save_data_to_array(max_len=feature_dim_2)


Saving vectors of label - '1': 100%|██████████| 126/126 [00:02<00:00, 48.92it/s]


In [25]:
X_train, X_test, y_train, y_test = get_train_test()
# # Feature dimension
feature_dim_1 = 20
channel = 1
epochs = 50
batch_size = 100
verbose = 1
num_classes = 8

# Reshaping to perform 2D convolution
X_train = X_train.reshape(X_train.shape[0], feature_dim_1, feature_dim_2, channel)
X_test = X_test.reshape(X_test.shape[0], feature_dim_1, feature_dim_2, channel)



In [28]:
def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
    model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='Adam',
                  metrics=['accuracy'])
    return model

In [29]:
model = get_model()
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(X_test, y_test))

Epoch 1/50
6/6 [==============================] - 2s 183ms/step - loss: 5.0135 - accuracy: 0.1499 - val_loss: 2.0975 - val_accuracy: 0.1478
Epoch 2/50
6/6 [==============================] - 1s 99ms/step - loss: 2.3788 - accuracy: 0.1586 - val_loss: 2.0940 - val_accuracy: 0.1935
Epoch 3/50
6/6 [==============================] - 1s 100ms/step - loss: 2.1000 - accuracy: 0.1788 - val_loss: 2.0346 - val_accuracy: 0.2392
Epoch 4/50
6/6 [==============================] - 1s 99ms/step - loss: 2.0417 - accuracy: 0.1906 - val_loss: 1.9992 - val_accuracy: 0.2500
Epoch 5/50
6/6 [==============================] - 1s 99ms/step - loss: 2.0113 - accuracy: 0.2059 - val_loss: 1.9723 - val_accuracy: 0.2849
Epoch 6/50
6/6 [==============================] - 1s 99ms/step - loss: 1.9980 - accuracy: 0.2215 - val_loss: 1.9414 - val_accuracy: 0.3199
Epoch 7/50
6/6 [==============================] - 1s 100ms/step - loss: 1.9452 - accuracy: 0.2549 - val_loss: 1.8949 - val_accuracy: 0.3253
Epoch 8/50
6/6 [========

In [50]:
def predict(filepath, model):
    sample = wav2mfcc(filepath)
    sample_reshaped = sample.reshape(1, feature_dim_1, feature_dim_2, channel)
    return get_labels()[0][
            np.argmax(model.predict(sample_reshaped))
    ]
testdata = os.listdir('/content/voice_dataset/test_set')
testdata.sort()
testdata.sort(key=lambda x:int(x[:-4]))
tt = pd.read_csv('//content/submission.csv')
test_target = tt['label']
test_predict = []



for a in testdata:
  test_predict.append(predict('/content/voice_dataset/test_set/' + a, model=model))




In [60]:
from sklearn import metrics 
predict = []
for a in test_predict:
  predict.append(int(a))


print(metrics.confusion_matrix(test_target, predict, labels=[0,1,2,3,4,5,6,7])) 
print(metrics.classification_report(test_target, predict)) 

[[ 88 137  90 105  68 130 123  81]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]]
              precision    recall  f1-score   support

           0       1.00      0.11      0.19       822
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0

    accuracy                           0.11       822
   macro avg       0.12      0.01      0.02       822
weighted avg       1.00      0.11      0.19       822



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
